#### Examples for machine learning algorithms by dsldPy

The goal is for users to train models with a simple, intuitive interface and also understand effects on fairness-utility tradeoffs based on hyperparamater selection. Examples are shown on training/testing sets with cross validation approaches.

1) regression examples using dsldPyFairML and dsldPyQeFairML
2) classification examples using dsldPyFairML and dsldPyQeFairML
3) k-fold cross validation to choose best hyperparameters for fairness utility tradeoff

In [1]:
# load necessary libraries
import pandas as pd
import numpy as np
import sys, os

# r-conversions
import rpy2.robjects as ro
from rpy2.robjects.packages import importr

# test accuracy
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, accuracy_score

# load dsld package
sys.path.append(os.path.abspath(".."))  

R = ro.r
R('if ("dsld" %in% loadedNamespaces()) detach("package:dsld", unload=TRUE)')
R('if (!requireNamespace("devtools", quietly=TRUE)) install.packages("devtools")')
R('devtools::load_all("~/Desktop/dsld", quiet=TRUE)')

from Utils import preprocess_data, read_data
from dsldPyFairML import (
    dsldPyFrrm, dsldPyFgrrm, dsldPyNclm, dsldPyZlm, dsldPyZlrm, dsldPyFairML_Summary, dsldPyFairML_Predict
)

from dsldPyQeFairML import (
    dsldPyQeFairKNN, dsldPyQeFairRF, dsldPyQeFairRidgeLin, dsldPyQeFairRidgeLog, dsldPyQeFairML_Predict
)

dsld = importr("dsld")


Error importing in API mode: ImportError("dlopen(/Users/adityamittal/miniconda3/lib/python3.12/site-packages/_rinterface_cffi_api.abi3.so, 0x0002): Library not loaded: /Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib\n  Referenced from: <1F2D8792-55A5-3398-8569-DDFF21A19C12> /Users/adityamittal/miniconda3/lib/python3.12/site-packages/_rinterface_cffi_api.abi3.so\n  Reason: tried: '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file)")
Trying to import in ABI mode.
R callback write-console: 




*********************



  Navigating dsld:

      Type vignette("Quick_Start",package="dsld") for a quick overview!

      Type vignette("Function_List",package="dsld") for a categorized functi

In [2]:
### regression example --- frrm(), nclm(), zlm(), qeFairKNN(), qeFairRF(), qeFairRidgeLin()

### read and preprocess data

### data preprocessing

### most dsldPy functions require a R data.frame object as input (NOT pandas dataframe)
### the preprocessing is done in the Utils.py file by the function preprocess_data
### user needs to manually provide the categorical and numerical features (list)
### the function preprocess_data returns a R data.frame object -> this is the required input for the dsldPy functions

# test and train split
df = read_data('~/Desktop/dsld/data/svcensus.RData')
test_df, train_df = train_test_split(df, test_size=0.3, random_state=42)
test_y = test_df['wageinc']
test_df = test_df.drop(columns=['wageinc'])

# preprocess data
cat_features_train = ['educ', 'occ', 'gender']
num_features_train = ['age', 'wageinc', 'wkswrkd']
svcensus_train = preprocess_data(train_df, cat_features_train, num_features_train)

cat_features_test = ['educ', 'occ', 'gender']
num_features_test = ['age', 'wkswrkd']
svcensus_test = preprocess_data(test_df, cat_features_test, num_features_test)


In [3]:
### using dsldPyFairML() function

### model training --- frrm() 
### unfairness = 0.05 // can also try different values for unfairness
a = dsldPyFrrm(svcensus_train, 'wageinc', 'gender', 0.05, definition = "sp-komiyama", lamda = 0, save = False)

# print train accuracy and correlations
print(f"train predictions: {a['train_predictions']}")
print(f"train accuracy: {a['train_accuracy']}")
print(f"train correlations: {a['train_correlations']}")

train predictions: [72300.94288981456, 60485.94790632362, 54039.50165443187, 73667.1176759253, 59045.680012413075, 31708.098759764976, 71607.35801304475, 46107.96443680182, 62855.75397272239, 72061.30303198629, 64704.79478547178, 96190.21934081278, 17908.581579332145, 79154.58635278385, 56485.67399866811, 56219.88321887697, 54585.8383104097, 89106.64482562144, 61176.04967409033, 65012.89387650562, 87849.13280903021, 59823.803115928386, 79839.41676978338, 82321.35540882153, 10073.279433766415, 74798.07592201183, 58106.01434662664, 74076.85807077396, 78351.5906253203, 53910.47395972161, 62303.52448287018, 71828.59496957944, 90512.21342205809, 64165.1455331317, 49708.41396020308, 96238.27131041388, 74066.705896357, 54363.21607514651, 57488.93264228231, 65970.10077309531, 30737.770166469134, 68516.94429057246, 64853.559382182044, 48294.473561048144, 64747.33258815921, 73977.08973908244, 62667.360946864435, 66406.15733428243, 59318.10615645327, 66425.84819882415, 74225.17089229995, 41678.52

In [4]:
### predict() on test data
a_preds = dsldPyFairML_Predict(a, svcensus_test)

# print test predictions and correlations
print(f"test predictions: {a_preds['test_predictions']}")
print(f"test correlations: {a_preds['test_correlations']}")

# manuallycompute test accuracy (MAPE)
test_accuracy = mean_absolute_error(test_y, a_preds['test_predictions'])
print(f"test accuracy: {test_accuracy}")

### the same can be done for other models --- nclm(), zlm() with dsldPyFairML_Predict() method

test predictions: [72389.22355133602, 57325.39078246606, 83349.46125982198, 59201.27247109954, 60598.253638357, 54766.850876145516, 75601.38002550432, 88631.28085817864, 34559.19644262316, 44937.09810249205, 92081.64737611973, 89055.249638072, 55843.85920631269, 71468.47957307754, 60714.26353600682, 8429.221373354732, 74193.0429887416, 93424.3655819499, 58334.117748717894, 62402.468082719766, 72650.70368657628, 54215.07584594913, 17507.103200759964, 6845.855829280297, 71623.84837314399, -8374.744013274963, 32244.24643525775, 69106.91333493136, 69113.99482017534, 57036.66902293963, 57437.045433786414, 6924.257259297598, 78873.41766238856, 95637.46547801007, 58486.897489882, 2626.6364437853, 33231.59601384817, 66818.64956649664, 77892.90331943055, 66446.7917993974, 12107.667972410685, 53711.13950484477, 88931.53730374806, 76213.99161622612, 60792.43780779416, 64687.79609014483, 78158.76652436792, 66818.58075202646, 75626.42281719822, 8010.725165059117, 37898.13483793622, 72774.9707741039

In [5]:
### using dsldPyQeFairML() functions 

### model training --- dsldQeFairRF() 
### deweightPars = {'educ': 0.2, 'occ': 0.05} // try different values for proxies
deweightPars = {'educ': 0.2, 'occ': 0.05}

a = dsldPyQeFairRF(svcensus_train, 'wageinc', 'gender', deweightPars)

# print train accuracy and correlations
print(f"train predictions: {a['train_predictions']}")
print(f"train accuracy: {a['train_accuracy']}")
print(f"train correlations: {a['train_correlations']}")

train predictions: [67245.3516763107, 62450.37695118711, 69334.70089422901, 85284.60725264168, 64233.81097359475, 289.2044083694083, 75335.80476701526, 39172.92418546489, 61521.533822697325, 79287.35613674788, 57042.778962648976, 91112.5905565843, 5.723809523809524, 79786.88944327403, 63088.96867531788, 43545.35060130021, 67997.01500719716, 62682.476833948866, 65483.53932390524, 72667.14639186658, 92566.54794764557, 63011.715337390764, 75247.38563314428, 94771.88323846912, 25.975174603174604, 76524.00296882045, 54397.305130871384, 76718.67984356612, 61563.082187775864, 36573.53429531336, 93132.99905694516, 78187.97779728888, 96765.78065565453, 84409.7464565463, 20637.369944073638, 90288.98220242508, 84035.85595720382, 48369.056316040784, 69468.25804575886, 70471.0626758383, 18654.512365762188, 58567.95138592924, 100824.26986332305, 65811.90535383455, 68477.3673374261, 79583.08856079828, 68903.54679223569, 71708.67716556277, 57303.30862032228, 70919.15882480124, 76237.39877402957, 46168

In [6]:
### predict on test data
a_preds = dsldPyQeFairML_Predict(a, svcensus_test)

# print test predictions and correlations
print(f"test predictions: {a_preds['test_predictions']}")
print(f"test correlations: {a_preds['test_correlations']}")

# manually compute test accuracy (MAPE)
test_accuracy = mean_absolute_error(test_y, a_preds['test_predictions'])
print(f"test accuracy: {test_accuracy}")

### the same can be done for other models --- qeFairKNN(), qeFairRidgeLin() with dsldPyQeFairML_Predict() method

test predictions: [88808.12599592653, 64845.016774596035, 85746.28475259831, 81668.19274613248, 58865.26837058438, 48925.81485099673, 70042.28281813425, 97656.2935560667, 32207.037627342663, 45156.33607543906, 100218.4910180394, 91782.68236395113, 59894.92845135348, 66505.8723656313, 62510.45093214162, 24475.83519347957, 86157.19887565704, 68079.9279493366, 68857.7717776639, 64513.769796571425, 76332.43976101326, 69754.77947713732, 19000.36143738821, 0.0, 76336.45298644698, 134.28627808699238, 29762.240953038632, 59223.84405462907, 60842.435541460676, 53856.37383658497, 61051.14144735088, 47.08079176711686, 57231.810824205044, 92326.83348053912, 58721.34574302633, 22081.235846440446, 33003.38756620256, 38262.72961385853, 67975.94920191563, 81547.42001028934, 1149.3417912066195, 62763.599158031844, 92639.81323134771, 82960.29065180093, 67268.35501726485, 67618.13869466895, 73614.86590226827, 67130.09828262482, 84732.31660195046, 24097.877571260935, 32782.6410354128, 57560.57700841383, 3

In [7]:
### classification examples --- fgrrm(), zlrm(), qeFairKNN(), qeFairRF(), qeFairRidgeLog()

### read and preprocess data

# test and train split
df = read_data('~/Desktop/dsld/data/compas1.RData')
test_df, train_df = train_test_split(df, test_size=0.3, random_state=42)
test_y = test_df['two_year_recid']
test_y = test_df['two_year_recid'].map({'Yes': 1, 'No': 0})            # convert to binary
test_df = test_df.drop(columns=['two_year_recid'])

# preprocess data
cat_features = ['sex', 'race', 'two_year_recid']
num_features = ["age", "juv_fel_count","decile_score","juv_misd_count","juv_other_count","priors_count","c_jail_in","c_jail_out","c_offense_date","screening_date","in_custody","out_custody"]
compas1_train = preprocess_data(train_df, cat_features_train, num_features_train)

cat_features = ['sex', 'race']
num_features = ["age", "juv_fel_count","decile_score","juv_misd_count","juv_other_count","priors_count","c_jail_in","c_jail_out","c_offense_date","screening_date","in_custody","out_custody"]
compas1_test = preprocess_data(test_df, cat_features_test, num_features_test)


In [8]:
### using dsldPyFairML() functions 

### model training --- fgrrm() 
### unfairness = 0.1 // try different values for unfairness
a = dsldPyFgrrm(compas1_train, 'two_year_recid', 'race', 0.1, definition = "sp-komiyama", family = "binomial", lamda = 0, save = False, yesYVal = "Yes")

# print train accuracy and correlations
print(f"train predictions: {a['train_predictions']}")             # returns prob = Yes
print(f"train accuracy (misclassification rate): {a['train_accuracy']}")
print(f"train correlations: {a['train_correlations']}")

train predictions: [0.6999988843595085, 0.7976134653951307, 0.8508285462523596, 0.4654643164347272, 0.4789612301910033, 0.25235891106523595, 0.3030127975816798, 0.2148297595633877, 0.9475174775547311, 0.40949938910478734, 0.45503935582797467, 0.08134039933351905, 0.22180403948605118, 0.750292134244151, 0.7996876838469726, 0.1993485369416968, 0.6257095658418004, 0.08412486609539203, 0.6849170671754015, 0.5073860633277777, 0.4169432691886657, 0.16487739793791134, 0.2699635038138084, 0.1098771274842079, 0.3223094747779274, 0.39017183631902036, 0.4734630488191124, 0.2529989214644707, 0.1626829048950782, 0.7932712294193632, 0.41423841413438706, 0.3158053844487543, 0.12170817071161427, 0.25205511052944407, 0.5872106821863736, 0.31330520386168254, 0.4155187700549714, 0.15074960993867606, 0.5138660135525559, 0.7762150103053489, 0.10257467995033202, 0.31132768458897975, 0.5334624196964795, 0.64770070926039, 0.49077201305648366, 0.5660051065876193, 0.6839213465270876, 0.7363365144115365, 0.07617

In [9]:
### predict() on test set
a_preds = dsldPyFairML_Predict(a, compas1_test)

# print test predictions and correlations
print(f"test predictions: {a_preds['test_predictions']}") # returns prob = Yes
print(f"test correlations: {a_preds['test_correlations']}")

# manually compute test accuracy (MAPE)
y_pred = [int(round(x)) for x in a_preds['test_predictions']]
test_accuracy = accuracy_score(test_y, y_pred)
misclass_rate = 1 - test_accuracy

# print train accuracy and correlations
print(f"test accuracy (misclassification rate): {misclass_rate}")

### the same can be done for other models --- zlrm() with dsldPyFairML_Predict() method

test predictions: [0.8816675673867042, 0.9271685936325658, 0.22173540355802504, 0.379970891225766, 0.2600193122583822, 0.7353566555367674, 0.5671591650769578, 0.6359607931545178, 0.27751302361478897, 0.7417992417323518, 0.39914806677327785, 0.7221148105464487, 0.3945160315602463, 0.24100569185035337, 0.5619804214713906, 0.5692574834566783, 0.34381200070113216, 0.8312701854283351, 0.29368945446559636, 0.7295570311438669, 0.974543942887882, 0.522347591418805, 0.6018998273644357, 0.1167618058432458, 0.356942641122594, 0.729283229035867, 0.7303367416173048, 0.1616571518102298, 0.24622168789717216, 0.444562314392117, 0.19066592902369872, 0.24904974333677363, 0.7247331502373162, 0.9419403629337713, 0.28454507048695754, 0.6961251869184233, 0.6221306999452251, 0.5486978972011496, 0.10490050747314368, 0.8010718307549787, 0.6832795162951995, 0.40419072355307256, 0.3816133397407489, 0.28173690482726493, 0.13126630387647958, 0.4201307695414147, 0.30239573704187317, 0.7095992161212672, 0.9846833484

In [10]:
### using dsldPyQeFairML() functions 

### model training --- dsldQeFairKNN() 
### deweightPars = {'decile_score': 0.2, 'priors_count': 0.5} // try different values for deweightPars
deweightPars = {'decile_score': 0.2, 'priors_count': 0.5}

a = dsldPyQeFairKNN(compas1_train, 'two_year_recid', 'race', deweightPars, k = 10, scaleX = True, yesYVal = "Yes")

# print train accuracy and correlations
# in the case of classification, the train_predictions returns both predClasses and prob = Yes
print(f"train predictions: {a['train_predictions']}")     
print(f"train accuracy: {a['train_accuracy']}")
print(f"train correlations: {a['train_correlations']}")

train predictions: [<rpy2.robjects.vectors.StrVector object at 0x173735450> [16]
R classes: ('character',)
['No', 'Yes', 'Yes', 'Yes', ..., 'No', 'No', 'No', 'No'], <rpy2.robjects.vectors.FloatVector object at 0x176b732d0> [14]
R classes: ('numeric',)
[0.400000, 1.000000, 1.000000, 0.600000, ..., 0.500000, 0.500000, 0.400000, 0.000000]]
train accuracy: 0.21798520204894706
train correlations: [('race==African-American', 0.14998156736483376), ('race==Caucasian', -0.09759215386570062), ('race==Other', -0.05914343456976784), ('race==Asian', -0.04729168300247519), ('race==Hispanic', -0.04209541612083901), ('race==Native American', 0.02215592574316403)]


In [11]:
### predict() on test set
a_preds = dsldPyQeFairML_Predict(a, compas1_test)

# print test predictions and correlations
print(f"test predictions: {a_preds['test_predictions']}")
print(f"test correlations: {a_preds['test_correlations']}")

# compute test accuracy
y_pred = [int(round(x)) for x in list(a_preds['test_predictions'][1])]
test_accuracy = accuracy_score(test_y, y_pred)
misclass_rate = 1 - test_accuracy

# print train accuracy and correlations
print(f"test accuracy (misclassification rate): {misclass_rate}")

### the same can be done for other models --- dsldQeFairRF(), dsldQeFairRidgeLog() with dsldPyQeFairML_Predict() method

test predictions: [<rpy2.robjects.vectors.StrVector object at 0x176b72c50> [16]
R classes: ('character',)
['Yes', 'No', 'No', 'No', ..., 'Yes', 'Yes', 'No', 'No'], <rpy2.robjects.vectors.FloatVector object at 0x176b73190> [14]
R classes: ('numeric',)
[1.000000, 0.500000, 0.500000, 0.200000, ..., 1.000000, 0.700000, 0.200000, 0.200000]]
test correlations: [('race==African-American', 0.1470458370617937), ('race==Caucasian', -0.09053050783972662), ('race==Hispanic', -0.06186680346724754), ('race==Other', -0.05209334589315818), ('race==Asian', -0.019529004080715857), ('race==Native American', -0.008734373971507456)]
test accuracy (misclassification rate): 0.27403611517813564


In [12]:
### k-fold cross validation to find best model based on fairness and accuracy